In [2]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim as gs
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')

In [5]:
train_dataset = pd.read_csv('../../train_data/train_data_1_1', delimiter=',')
# validation_dataset = pd.read_csv('../train_data/validation_data_1_1', delimiter=',')

In [ ]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [ ]:
MAX_PAD_TITLE = 55

def sequence_padding_title(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_TITLE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding_title(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_TITLE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding_title(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_TITLE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
MAX_PAD_MAIN_PASSAGE = 55

def sequence_padding_main_passage(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_MAIN_PASSAGE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding_main_passage(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_MAIN_PASSAGE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding_main_passage(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_MAIN_PASSAGE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
train_article_title = []
train_article_main_passage = []
train_table_title = []
train_table_main_passage = []
train_label = []

for i,row in tqdm(train_dataset.iterrows()):
    
    train_article_title.append(create_embedding_title(row['article_page_title']))
    train_table_title.append(create_embedding_title(row['table_page_title']))
    
    train_article_main_passage.append(create_embedding_main_passage(row['article_meta_description']))
    train_table_main_passage.append(create_embedding_main_passage(row['table_page_summary']))
    
    train_label.append(row['label'])

In [ ]:
train_article_title = np.array(train_article_title, dtype='float16')
train_table_title = np.array(train_table_title, dtype='float16')
train_article_main_passage = np.array(train_article_main_passage, dtype='float16')
train_table_main_passage = np.array(train_table_main_passage, dtype='float16')
train_label = np.array(train_label)

In [ ]:
train_article_main_passage.shape

In [ ]:
validation_article_title = []
validation_article_main_passage = []
validation_table_title = []
validation_table_main_passage = []
validation_label = []

for i, row in tqdm(validation_dataset.iterrows()):
    
    validation_article_title.append(create_embedding_title(row['article_page_title']))
    validation_table_title.append(create_embedding_title(row['table_page_title']))  
    
    validation_article_main_passage.append(create_embedding_main_passage(row['article_meta_description']))
    validation_table_main_passage.append(create_embedding_main_passage(row['table_page_summary']))
    
    validation_label.append(row['label'])

In [ ]:
validation_article_title = np.array(validation_article_title)
validation_article_main_passage = np.array(validation_article_main_passage)
validation_table_title = np.array(validation_table_title)
validation_table_main_passage = np.array(validation_table_main_passage)
validation_label = np.array(validation_label)

In [ ]:
validation_article_main_passage.shape

In [ ]:
def affinity_method(query, document):
    
    transpose_document = tf.transpose(document,perm=[0, 2, 1])
    
    affinity = tf.matmul(query,transpose_document)
    
    normalize_affinity = tf.nn.softmax(affinity)
    
    return normalize_affinity

In [ ]:
def attention_model():
    
    # inputs
    article_title = tf.keras.Input(shape=(55,50), dtype='float32')
    article_main_passage = tf.keras.Input(shape=(55,50), dtype='float32')
    table_title = tf.keras.Input(shape=(55,50), dtype='float32')
    table_main_passage = tf.keras.Input(shape=(55,50), dtype='float32')
    
    # article and tables signal representations
    rep_article_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_title)
    rep_article_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_main_passage)
    rep_table_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_title)
    rep_table_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_main_passage)
    
    
    #affinity article title and table title over two spaces
    aff_A_title_T_title1 = affinity_method(rep_article_title,rep_table_title)
    rep_aff_A_title_T_title1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_title_T_title1)
    vector_A_title_T_title1 = tf.keras.layers.Flatten()(rep_aff_A_title_T_title1)
    
    aff_A_title_T_title2 = affinity_method(article_title,table_title)
    rep_aff_A_title_T_title2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_title_T_title2)
    vector_A_title_T_title2 = tf.keras.layers.Flatten()(rep_aff_A_title_T_title2)
    
    #affinity article title and table text over two spaces
    aff_A_title_T_text1 = affinity_method(rep_article_title,rep_table_text)
    rep_aff_A_title_T_text1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_title_T_text1)
    vector_A_title_T_text1 = tf.keras.layers.Flatten()(rep_aff_A_title_T_text1)
    
    aff_A_title_T_text2 = affinity_method(article_title,table_main_passage)
    rep_aff_A_title_T_text2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_title_T_text2)
    vector_A_title_T_text2 = tf.keras.layers.Flatten()(rep_aff_A_title_T_text2)
    
    #affinity article text and table title over two spaces
    aff_A_text_T_title1 = affinity_method(rep_article_text,rep_table_title)
    rep_aff_A_text_T_title1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_text_T_title1)
    vector_A_text_T_title1 = tf.keras.layers.Flatten()(rep_aff_A_text_T_title1)
    
    aff_A_text_T_title2 = affinity_method(article_main_passage,table_title)
    rep_aff_A_text_T_title2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_text_T_title2)
    vector_A_text_T_title2 = tf.keras.layers.Flatten()(rep_aff_A_text_T_title2)
    
    #affinity article text and table text over two spaces
    aff_A_text_T_text1 = affinity_method(rep_article_text,rep_table_text)
    rep_aff_A_text_T_text1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_text_T_text1)
    vector_A_text_T_text1 = tf.keras.layers.Flatten()(rep_aff_A_text_T_text1)
    
    aff_A_text_T_text2 = affinity_method(article_main_passage,table_main_passage)
    rep_aff_A_text_T_text2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(aff_A_text_T_text2)
    vector_A_text_T_text2 = tf.keras.layers.Flatten()(rep_aff_A_text_T_text2)
    
    #learned vector
    final_vector = tf.keras.layers.Concatenate()([vector_A_title_T_title1,vector_A_title_T_title2,vector_A_title_T_text1,vector_A_title_T_text2,vector_A_text_T_title1,vector_A_text_T_title2,vector_A_text_T_text1,vector_A_text_T_text2])
    
    MLP_input = tf.keras.layers.Dense(128, activation='relu')(final_vector)
    dropout1 = tf.keras.layers.Dropout(0.5, name="dropout1")(MLP_input)
    MLP_hidden = tf.keras.layers.Dense(64, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.5, name="dropout2")(MLP_hidden)
    MLP_output = tf.keras.layers.Dense(1,activation='sigmoid')(dropout2)
    
    model = tf.keras.Model(inputs=[article_title,article_main_passage,table_title,table_main_passage],outputs=MLP_output)
    
    return model

In [ ]:
model = attention_model()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=['accuracy'])

In [ ]:
filepath="affinity_model_title_main_passage_{epoch:02d}_{val_accuracy:.4f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit([train_article_title, train_article_main_passage, train_table_title, train_table_main_passage], train_label, 
          epochs=20, 
          batch_size=32, 
          verbose=2,
          validation_data=([validation_article_title, validation_article_main_passage, validation_table_title, validation_table_main_passage], validation_label),
          callbacks=callbacks_list)